## Fist Time Setup Dont Touch

### Bulk read All Files
Read all pdf files stored in `BANK_STATMENT` on a drop box and re-analyse them

In [ ]:
# bulk read (only do this)
import os
import pdf_bank_statement.analyser as analyser
import pdf_bank_statement.reader as reader

BANK_STATEMENT = "/Users/frankliang/Dropbox/Apps/Bank-Statement-Reader/BANK_STATEMENT"
all_files = os.listdir(BANK_STATEMENT)

all_statement=[*map(lambda x: BANK_STATEMENT + '/' + x, all_files)]


df = reader.read(all_statement)
analysed=analyser.analyse(df) # categorise mainly

Read... Total 1 tables populated.
Read... Total 2 tables populated.
Read... Total 3 tables populated.
Read... Total 4 tables populated.
Read... Total 5 tables populated.
Read... Total 6 tables populated.
Read... Total 7 tables populated.
Read... Total 8 tables populated.
Read... Total 9 tables populated.
Read... Total 10 tables populated.
Read... Total 11 tables populated.
cleaning complete.


### First Time Bluk Create a Table

Never run this script after everything has been set up. Because we will 
use reference number id to store which transaction are marked as varible, 
which one are none variable.


**Concerns For Multiple Bank Account**

Before we do so we must consider for point where we may actually open different bank statement or we end up unable to tell which account is which...

So it maybe worth creating a foregin key on a column. Add additional table here [Reference](https://www.devart.com/dbforge/sql/studio/add-column-to-table-sql-server.html#:~:text=The%20basic%20syntax%20of%20the,to%20add%20a%20new%20column.) 

In [ ]:
import duckdb
im_con = duckdb.connect()
analysed_data = analysed.data
analysed_data.sample(3)
duckdb.from_df(analysed_data.iloc[:,0:8].sample(3))

┌─────────────────────┬────────────────────┬─────────┬───┬─────────────┬────────────────┬──────────────────────┐
│        Date         │    Description     │  Type   │ … │ Balance (£) │    Category    │       src_file       │
│    timestamp_ns     │      varchar       │ varchar │   │   double    │    varchar     │       varchar        │
├─────────────────────┼────────────────────┼─────────┼───┼─────────────┼────────────────┼──────────────────────┤
│ 2023-01-19 00:00:00 │ CO-OP GROUP 070527 │ DEB     │ … │      2597.0 │ Food & Grocery │ /Users/frankliang/…  │
│ 2023-02-28 00:00:00 │ TESCO STORES 2487  │ DEB     │ … │      4642.0 │ Food & Grocery │ /Users/frankliang/…  │
│ 2023-10-02 00:00:00 │ Zettle_*Sundays    │ DEB     │ … │      5663.0 │ Other          │ /Users/frankliang/…  │
├─────────────────────┴────────────────────┴─────────┴───┴─────────────┴────────────────┴──────────────────────┤
│ 3 rows                                                                                   8 col

In [ ]:
import duckdb

DB_PATH = '/Users/frankliang/Dropbox/Apps/DuckDb/personal.db'
with duckdb.connect(DB_PATH) as con:
    try:
        con.sql('''
                DROP TABLE my.bank_statement;
                DROP SEQUENCE seq_bank_statement;
                ''')
        print("Table droped")
    except:
       pass
    finally:
        print("Creating new table")
        con.sql('''
                CREATE SCHEMA IF NOT EXISTS my;
                CREATE SEQUENCE IF NOT EXISTS seq_bank_statement START 1;
                CREATE TABLE my.bank_statement (
                    Id INTEGER PRIMARY KEY,
                    Date timestamp_ns,
                    Description VARCHAR,
                    Type VARCHAR(3),
                    "Money In (£)" DOUBLE,
                    "Money Out (£)" DOUBLE,
                    "Balance (£)" DOUBLE,
                    "Category" VARCHAR(32),
                    src_file VARCHAR(100)
                );
                INSERT INTO my.bank_statement
                SELECT nextval('seq_bank_statement'),* 
                FROM analysed_data
        ''')
        print("Sampled Data Looks Like this")
        print(con.sql("Select * from my.bank_statement order by random() limit 3"))

    